In [ ]:
# parse

In [1]:
import json
import pandas as pd
import os
import shutil

DATA_DIR = "./data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "store_tag_dump.pkl")

with open(DATA_FILE, encoding="utf-8") as f:
    data = json.loads(f.read())

In [2]:
store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도
    "longitude",  # 음식점 경도
    "category",  # 음식점 카테고리
)

menu_columns = (
    "id",            # 메뉴 고유번호
    "store",         # 음식점 고유번호
    "menu_name",     # 메뉴 이름
    "price",         # 가격
)

user_columns = (
    "id",            # 유저 고유번호
    "gender",        # 성별
    "born_year",     # 생년
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

In [3]:
stores = []   # 음식점 테이블
menus = []    # 메뉴 테이블
users = set()  # 유저 테이블
reviews = []  # 리뷰 테이블

mid = 1

for d in data:

    categories = [c["category"] for c in d["category_list"]]
    stores.append(
        [
            d["id"],
            d["name"],
            d["branch"],
            d["area"],
            d["tel"],
            d["address"],
            d["latitude"],
            d["longitude"],
            "|".join(categories),
        ]
    )

    for menu in d["menu_list"]:
        # 메뉴 고유번호(id), 음식점 고유번호(store), 메뉴 이름(menu_name), 가격(price)
        menus.append([mid, d["id"], menu["menu"], menu["price"]])
        mid += 1

    for review in d["review_list"]:
        r = review["review_info"]
        u = review["writer_info"]

        reviews.append(
            [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
        )

        users.add(
            (u["id"], u["gender"], u["born_year"])
        )


store_frame = pd.DataFrame(data=stores, columns=store_columns)
menu_frame = pd.DataFrame(data=menus, columns=menu_columns)
user_frame = pd.DataFrame(data=users, columns=user_columns)
review_frame = pd.DataFrame(data=reviews, columns=review_columns)

In [4]:
# make new store 

In [5]:
stores = pd.merge(store_frame, review_frame,left_on="id",right_on="store", how="left")
stores = stores.groupby(["id_x","store_name"]).mean()
stores = pd.merge(stores,store_frame,left_on="id_x",right_on="id")

In [6]:
stores.head()

,id_y,store,user,score,id,store_name,branch,area,tel,address,latitude,longitude,category
0,NaN,NaN,NaN,NaN,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차
1,NaN,NaN,NaN,NaN,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페
2,NaN,NaN,NaN,NaN,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍
3,NaN,NaN,NaN,NaN,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제
4,NaN,NaN,NaN,NaN,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트


In [7]:
# category가 없는 가게들
stores = stores[stores['category'] != '']

In [8]:
stores.head()

,id_y,store,user,score,id,store_name,branch,area,tel,address,latitude,longitude,category
0,NaN,NaN,NaN,NaN,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차
1,NaN,NaN,NaN,NaN,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페
2,NaN,NaN,NaN,NaN,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍
3,NaN,NaN,NaN,NaN,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제
4,NaN,NaN,NaN,NaN,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트


In [9]:
len(stores)

276537

In [10]:
# 카테고리 재생성

In [11]:
from konlpy.tag import Komoran

In [12]:
info = {'0': [],
    '1':["떡볶이","순대","라볶이"],
    '2': ["피자"],
    '3':['치킨','통닭','양념치킨','닭다리','후라이','파닭','크리스'],
    '4':['모밀','돈카츠','돈가스','돈까스','스시','도쿄라멘','메로','스키야키','참치','횟집','라멘','송어','삼치','문어','해삼','우럭','꼼장어','새우','참다랑어','고래고기','붕장어','꽁치','산천어','초밥','장어','낙지','대개','전복','조개','대하','방어','도미','회덮밥','풍천장어','우동','오코노','미야','광어','사시미','민물고기','감성돔','홍어','오니기리','회전','나가사끼','나가사키','소바','라멘','규동','미소라멘','돈부리','일본'],
    '5':['츄러스','베이커리','카페','홍차','커피','밀크티','마카롱','북카페','다쿠아즈','쿠키','고로케','빙수','파이','타르트','호두과자','마들렌','아메리카노','아이스크림','브레드','쥬스','브라우니','디저트','스콘','도넛','롤케이크','커피숍','티라미수','슈크림','빵집','머랭','케익','쇼콜라','바닐라','카푸치노','누텔라','만화','망고','크림','로스팅','와플','토스트','식빵','초콜렛','녹차','베이글','버블티','찻집','딸기','빼빼로','다방','떡집','더치','자몽','스터디','꽈배기','수플레','고양이','바게트','치즈케이크','컵케익','말차','파운드','티라미스','자전거','딸기우유','단팥빵','모찌','제과점','붕어빵','치아바타','소프트','과자','호떡','에스프레소','팬케이크','핸드','파르페','카페모카','다쿠아즈','통밀빵','프레첼','인절미','에이드','스무디','찹쌀떡','푸딩'],
    '6':['쌀국수','샤브샤브','월남쌈','아시아','베트','케밥','팟타이','카레','대만','커리','인도','중국','터키','라씨','태국','라쟈냐','밀푀유','딤섬','타코','싱가포르'],
    '7':['랍스터','멕시코','브라질','호주','퓨전','고르곤졸라','러시아','스테이크','프랑스','이탈라인','스파게티','브런치','벨기에','샐러드','파스타','레스토랑','양식','하와이','그리스','뉴욕','케냐','맥시칸','앵거스','오믈렛','함박','파니니','이태리','스페인','스튜','필라프','에그','햄버그','아보카도','독일','아스파라거스','라따뚜이','케밥','감바스','미국','드라이','이징','체코','웨지감자','칠리','바비큐','타파스'],
    '8':['짬뽕','중국','짜장면','중화요리','깐풍기','중국집','탕수육','훠궈','마파두부','홍콩','수타면','양고기','마라','샤오롱바오','사천','마라탕','자장면','군만두','오징어짬뽕'],
    '9':['샌드위치','도시락'],
    '10':['버거','치즈버거','치즈스틱'],
    '11':['펍','비어','소주','술집','맥주','이자카야','와인','카스','동동주','테라','기네스','전통주','선술집','흑맥주','포차','막걸리','치맥','칵테일','파전','생맥주','호프','위스키','포장마차','민속주','비어','전집','주점','보드카','에일'],
    '12':['족발','보쌈'],
    '13':['갈비찜','아구찜','안동찜닭','아귀찜','장탕','갈비탕','보신탕','닭볶음탕','도가니탕','곰탕','해물탕','설렁탕','뼈다귀','등뼈','내장','감자탕']
    }

In [13]:
category_data= []
no_category = []
only_store_name = []
km = Komoran()
words = {}
add_list = []

In [14]:
for index, store in stores.iterrows():
    category = store.category
    category.encode('utf-8')
    
    try:

        nouns = km.nouns(category)

        if not nouns:
            if category:
                v_flag = True
                for key, value in info.items():
                    if v_flag:
                        for v in value:
                            if v in category:
                                category_data.append({
                                    "store_id": store.id,
                                    "category": int(key)
                                })
                                add_list.append(int(key))
                                v_flag = False
                                break

                if v_flag:            
                    category_data.append({
                        "store_id" : store.id,
                        "category" : 0,
                    })
                    add_list.append(int(0))
                    continue
            else:
                if store.score >= 3.5 :
                    no_category.append({
                        "store_id" : store.id,
                        "store_name" : store.name,
                    })
            continue

    except:
        continue

    # 한식인지 구분위한 flag
    flag = True
    if not nouns:
        continue
        
    for key, value in info.items():
        if flag:
            for v in value:
                if v in nouns:
                    category_data.append({
                        "store_id": store.id,
                        "category": int(key)
                    })
                    add_list.append(int(key))
                    flag = False
                    break
    if flag:
        category_data.append({
            "store_id": store.id,
            "category": 0,
        })
        add_list.append(int(0))

In [15]:
len(stores)

276537

In [16]:
len(add_list)

276537

In [17]:
stores['new_category'] = add_list

In [18]:
stores.head()

,id_y,store,user,score,id,store_name,branch,area,tel,address,latitude,longitude,category,new_category
0,NaN,NaN,NaN,NaN,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,11
1,NaN,NaN,NaN,NaN,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,5
2,NaN,NaN,NaN,NaN,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,11
3,NaN,NaN,NaN,NaN,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,7
4,NaN,NaN,NaN,NaN,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,5


In [19]:
stores = stores[['id', 'store_name', 'branch', 'area', 'tel', 'address', 'latitude', 'longitude', 'new_category']]

In [20]:
stores.head(n=10)

,id,store_name,branch,area,tel,address,latitude,longitude,new_category
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,11
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,5
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,11
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,7
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,5
5,6,Gogoss Pub,None,하남,031-792-9289,경기도 하남시 덕풍동 412-18,37.539046,127.203933,11
6,7,Madeleine,None,만촌동,053-759-7947,대구광역시 수성구 만촌1동 1356-7,35.869088,128.636058,5
8,9,Red Camp,None,진주,070-7786-9258,경상남도 진주시 평거동 792-3,35.170957,128.064554,0
9,10,T원,None,신촌,02-3145-2904,서울특별시 서대문구 창천동 30-33,37.556074,126.935798,7
10,11,국수이야기,None,월곶,031-317-8002,경기도 시흥시 월곶동 1006-3,37.388700,126.740000,0


In [21]:
store_frame.head(n=10)

,id,store_name,branch,area,tel,address,latitude,longitude,category
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트
5,6,Gogoss Pub,None,하남,031-792-9289,경기도 하남시 덕풍동 412-18,37.539046,127.203933,펍
6,7,Madeleine,None,만촌동,053-759-7947,대구광역시 수성구 만촌1동 1356-7,35.869088,128.636058,마들렌|베이커리
7,8,NesCafe,None,남포동,051-243-1125,부산광역시 중구 광복동2가 25,35.100110,129.032478,
8,9,Red Camp,None,진주,070-7786-9258,경상남도 진주시 평거동 792-3,35.170957,128.064554,고깃집|고기집
9,10,T원,None,신촌,02-3145-2904,서울특별시 서대문구 창천동 30-33,37.556074,126.935798,중식당|레스토랑


In [22]:
review_frame

,id,store,user,score,content,reg_time
0,1,15,68632,5,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,1970-01-01 00:00:00
1,2,18,389728,5,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,1970-01-01 00:00:00
2,3,19,68716,4,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,1970-01-01 00:00:00
3,4,37,774353,2,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",1970-01-01 00:00:00
4,5,38,115682,3,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2019-03-15 22:16:47
...,...,...,...,...,...,...
91393,2401,360499,17371,5,,1970-01-01 00:00:00
91394,2402,360505,198050,4,닭발이 소스가 매운데 중독성있는 맛입니다. 마지막에 볶음밥 꼭드세요,1970-01-01 00:00:00
91395,2403,360514,190766,5,늘 사람 많은 곳입니다. 소고기만 먹어도 배불러요! 굿,1970-01-01 00:00:00
91396,2404,360514,201564,4,고기가 엄청많이 먹고싶을 때 종종 들리는 곳이에요\n뷔페처럼 샐러드바에 사이드메뉴가...,1970-01-01 00:00:00


In [23]:
# 태그 생성

In [24]:
tag_data = []
tag_dict = {"가성비":0,"저렴":0,"깔끔":1,"신선":1,"위생":1,"깨끗":1,"친절":2,"분위기":3,"감성":3,"인테리어":4,"아침":5,"조식":5,"새벽":5,"점심시간":6,"점심":6,"낮":6,"퇴근":7,"석식":7,"술":7,"야식":7,"밤":7,"저녁":7,"친구":8,"연인":9,"데이트":9,"둘이서":9,"여자친구":9,"여친":9,"남자친구":9,"남친":9,"커플":9,"가족":10,"부모님":10,"주차장":11,"주차공간":11,"주차권":11}
reviews = review_frame
km = Komoran()
words = {}

In [25]:
test = reviews[["content","score","store"]]
store_tags = pd.merge(stores, test, left_on="id", right_on="store", how="left")

In [26]:
store_tags.head(n=20)

,id,store_name,branch,area,tel,address,latitude,longitude,new_category,content,score,store
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,11,NaN,NaN,NaN
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,5,NaN,NaN,NaN
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,11,NaN,NaN,NaN
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,7,NaN,NaN,NaN
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,5,NaN,NaN,NaN
5,6,Gogoss Pub,None,하남,031-792-9289,경기도 하남시 덕풍동 412-18,37.539046,127.203933,11,NaN,NaN,NaN
6,7,Madeleine,None,만촌동,053-759-7947,대구광역시 수성구 만촌1동 1356-7,35.869088,128.636058,5,NaN,NaN,NaN
7,9,Red Camp,None,진주,070-7786-9258,경상남도 진주시 평거동 792-3,35.170957,128.064554,0,NaN,NaN,NaN
8,10,T원,None,신촌,02-3145-2904,서울특별시 서대문구 창천동 30-33,37.556074,126.935798,7,NaN,NaN,NaN
9,11,국수이야기,None,월곶,031-317-8002,경기도 시흥시 월곶동 1006-3,37.388700,126.740000,0,NaN,NaN,NaN


In [43]:
len(stores)

276537

In [42]:
len(store_tags)

321880

In [44]:
review_id = -1
tag_lists = []

for index, review in store_tags.iterrows():
    if review_id != review['id']:
        if review_id != -1:
            tag_lists.append(','.join(map(str,list(tags))))
        tags = set()
        review_id = review['id']
    try:
        nouns = km.nouns(review.content)
    except:
        pass
    for noun in nouns:
        if noun in tag_dict.keys() :
            if review.score >= 4 :
                tags.add(tag_dict[noun])

        if noun not in words:
            words[noun] = 0
        words[noun] += 1

    if tags :
        data = {
                "store_id" : review.store,
                "tags":list(tags)
                    }
        tag_data.append(data)
tag_lists.append(','.join(map(str,list(tags))))

In [47]:
len(tag_lists)

276537

In [48]:
len(store_tags)

321880

In [49]:
stores['tags'] = tag_lists

In [60]:
len(stores[stores['tags'] != ''])

13586

In [57]:
pd.to_pickle(stores, DUMP_FILE)

In [59]:
stores.head(n=50)

,id,store_name,branch,area,tel,address,latitude,longitude,new_category,tags
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,11,
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,5,
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,11,
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,7,
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,5,
5,6,Gogoss Pub,None,하남,031-792-9289,경기도 하남시 덕풍동 412-18,37.539046,127.203933,11,
6,7,Madeleine,None,만촌동,053-759-7947,대구광역시 수성구 만촌1동 1356-7,35.869088,128.636058,5,
8,9,Red Camp,None,진주,070-7786-9258,경상남도 진주시 평거동 792-3,35.170957,128.064554,0,
9,10,T원,None,신촌,02-3145-2904,서울특별시 서대문구 창천동 30-33,37.556074,126.935798,7,
10,11,국수이야기,None,월곶,031-317-8002,경기도 시흥시 월곶동 1006-3,37.388700,126.740000,0,
